In [2]:
import pandas as pd
import numpy as np
import math
from collections import Counter
import os
import matplotlib.pyplot as plt
from scipy.stats import linregress
from sklearn.svm import SVC
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [4]:
sensor_data = pd.DataFrame.from_csv("../../data/database/sensor_data.csv")
sensor_data = sensor_data[~((sensor_data.TagName == 'Start1') | (sensor_data.TagName == 'Start2'))]
Counter(sensor_data.TagName)

Counter({'Tag0': 352461,
         'Tag1': 272234,
         'Tag2': 377420,
         'Tag3': 398370,
         'Tag4': 428642,
         'Tag5': 435086,
         'Tag6': 358288,
         'Tag7': 315266,
         'Tag8': 431390,
         'Tag9': 381880})

In [5]:
sensor_data_acc = sensor_data[(sensor_data.SENSORTYPE == 1)]
# tag_id_groupby = sensor_data_acc_tag12.groupby(['TagName', 'tester_id'])
tag_id_groupby_acc = sensor_data_acc.groupby(['TagName', 'tester_id'])
tag_id_dict = tag_id_groupby_acc.groups
y = list(tag_id_dict.keys())
y = [i[0] for i in y]
y = np.array(y)

sensor_data_magnetic = sensor_data[(sensor_data.SENSORTYPE == 2)]
tag_id_groupby_magnetic = sensor_data_magnetic.groupby(['TagName', 'tester_id'])

sensor_data_orientation = sensor_data[(sensor_data.SENSORTYPE == 3)]
tag_id_groupby_orientation = sensor_data_orientation.groupby(['TagName', 'tester_id'])

sensor_data_gyro = sensor_data[(sensor_data.SENSORTYPE == 4)]
tag_id_groupby_gyro = sensor_data_gyro.groupby(['TagName', 'tester_id'])

sensor_data_gravity = sensor_data[(sensor_data.SENSORTYPE == 9)]
tag_id_groupby_gravity = sensor_data_gravity.groupby(['TagName', 'tester_id'])

sensor_data_quaternion = sensor_data[(sensor_data.SENSORTYPE == 11)]
tag_id_groupby_quaternion = sensor_data_quaternion.groupby(['TagName', 'tester_id'])

sensor_data_tilt = sensor_data[(sensor_data.SENSORTYPE == 26)]
tag_id_groupby_tilt = sensor_data_tilt.groupby(['TagName', 'tester_id'])

In [11]:
# there will be 9 frames, frame0 to frame8
# There will be 10 segments
N_frame_no = 15

test_percent = 0.1 # 10% samples are used for testing

In [32]:
def gesture_features(accs):
#     for i in range(accs.shape[0]): # x, y, z
#         accs[i] = min_max_normalization(accs[i], np.min(accs[i]), np.max(accs[i]))
        
    if N_frame_no > 1:
        Ls = math.floor(len(accs)/ (N_frame_no + 1))
        segments = None
        for i in range(N_frame_no + 1):
            if segments is None:
                segments = np.array([accs[i*Ls:(i+1)*Ls]])
            else:
                segments = np.append(segments, np.array([accs[i*Ls:(i+1)*Ls]]), axis=0)

        frames = None
        for i in range(N_frame_no):
            cur_frame = segments[i:i+2]
            cur_frame = cur_frame.reshape((cur_frame.shape[0]*cur_frame.shape[1],cur_frame.shape[2]))
            if frames is None:
                frames = np.array([cur_frame])
            else:
                frames = np.append(frames, np.array([cur_frame]), axis = 0)

        
        return np.array([frame_features(f) for f in frames])
    else:
        return frame_features(accs).reshape(-1)

In [33]:
def frame_features(cur_frame):
    dft_cur_frame = np.fft.fftn(cur_frame)
    
    mean_cur_frame = dft_cur_frame[0]

    energy_cur_frame=[]
    for T in range(cur_frame.shape[1]): #x,y,z
        T_sum = 0
        for i in range(1,len(cur_frame)):
            T_sum += math.pow(abs(dft_cur_frame[i,T]),2)

        energy_cur_frame.append(T_sum / (len(cur_frame)-1))
    energy_cur_frame = np.array(energy_cur_frame)
    
    entropy_cur_frame = []
    for T in range(cur_frame.shape[1]): #x,y,z
        T_sum = 0
        entropy_sum = 0
        for i in dft_cur_frame[1:,T]:
            T_sum += abs(i)
            
        for m in dft_cur_frame[1:,T]:
            p_m_T = abs(m)/T_sum
            entropy_sum += p_m_T*math.log(1/p_m_T)

        entropy_cur_frame.append(entropy_sum)
    entropy_cur_frame = np.array(entropy_cur_frame)
    

    std_cur_frame = []
    for T in range(cur_frame.shape[1]): #x,y,z
        std_cur_frame.append(np.std(cur_frame))
    std_cur_frame = np.array(std_cur_frame)
    
    coorelation_cur_frame = []
    for T1,T2 in [(0,1),(1,2),(0,2)]:
        coorelation_cur_frame.append(np.correlate(cur_frame[:,T1], cur_frame[:,T2])[0])
    coorelation_cur_frame = np.array(coorelation_cur_frame)
    
    return np.concatenate((mean_cur_frame, energy_cur_frame, entropy_cur_frame, std_cur_frame))

In [34]:
def shrink_array(array,size):
    
    ratio = float(len(array)) / float(size+1)
    res = []
    for i in range(size):
        res.append(np.mean(array[math.floor(i*ratio):math.ceil((i+1.0)*ratio)], axis = 0))
    return np.array(res)

In [35]:
X = []
for key in list(tag_id_dict.keys()):
    frame_feature = gesture_features(tag_id_groupby_acc.get_group(key)[['VALUES1', 'VALUES2', 'VALUES3']].values)
#     acc_feature = shrink_array(tag_id_groupby_acc.get_group(key)[['VALUES1','VALUES2', 'VALUES3']].values, 200).reshape(-1)
#     t = pd.to_datetime(tag_id_groupby_acc.get_group(key)['TIMESTAMP']).values
#     time_dif = (np.max(t) - np.min(t)).item()/1000000000
    X.append(frame_feature)
#     X.append(np.concatenate((acc_feature, np.array([time_dif]))))
X = np.array(X)

In [36]:
X.shape

(1070, 15, 12)

In [37]:
# idx_list = list(range(len(X)))
# np.random.shuffle(idx_list)
# train_idx = idx_list[:math.floor(len(X) * (1- test_percent))]
# test_idx = idx_list[math.floor(len(X) * (1- test_percent)):]

# # y[y == 'Tag2'] = 2
# # y[y == 'Tag1'] = 1

# train_x = X[train_idx]
# train_y = y[train_idx]
# test_x = X[test_idx]
# test_y = y[test_idx]

clf4 = SVC(kernel='poly', C=1, degree=3, verbose = True)
# clf4.fit(train_x, train_y) 
# joblib.dump(clf4, '../../Results/baseline SVC 0.80 raw data acc with gyro 200 chunk.pkl') 
# print(classification_report(test_y, clf4.predict(test_x)))
if len(X.shape) > 2:
    res = cross_val_score(clf4, X.reshape(list(X.shape)[0],-1), y, cv = 5)
else:
    res = cross_val_score(clf4, X, y, cv=5)
print(res)
print(np.mean(res))

/Users/Ageha/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:382: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)
/Users/Ageha/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:382: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


[LibSVM][LibSVM]

/Users/Ageha/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:382: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)
/Users/Ageha/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:382: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


[LibSVM][LibSVM][LibSVM][ 0.61818182  0.61818182  0.49047619  0.68095238  0.5       ]
0.581558441558


/Users/Ageha/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:382: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)
